In [1]:
import pandas as pd
import re
import time
# from nltk.corpus import words
from nltk.tokenize import word_tokenize
import nltk
#nltk.download('punkt')
from textblob import Word
pd.options.display.width = 0

import spacy
nlp = spacy.load('en_core_web_sm')
all_stopwords = nlp.Defaults.stop_words

pd.set_option('display.max_rows', 6000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)
#pd.describe_option('display')
# import enchant

In [2]:
past_time = time.time()

In [3]:
path='/nfs/home/vyasa/projects/proj_off/data_off/clarify/spanish_comorbidity/new/'
path_output = '/nfs/home/vyasa/projects/proj_off/data_off/clarify/spanish_comorbidity/new/output/'
    

In [4]:
df = pd.read_csv(path+'comorbidities_modified_witht_abbr_new.csv', sep=',', error_bad_lines=True, encoding='utf-8')
df.shape

(5451, 4)

In [5]:
df.head(5)

,Value_ID,value,value_witht_abbr,replacement
0,A10000,Gastritis crÛnica,Gastritis crÛnica,Chronic gastritis;
1,A10001,Sarcoma PleomÛrfico Pectoral Mayor,Sarcoma PleomÛrfico Pectoral Mayor,Pectoralis major pleomorphic sarcoma;
2,A10002,hiperuricemia,hiperuricemia,Hyperuricemia;
3,A10003,intervenido pÛlipo colon,intervenido pÛlipo colon,Colon polyp operated;
4,A10004,intervenido estenosis aorta,intervenido estenosis aorta,Aortic stenosis operated;


In [6]:
df['IsValueReplaced'] = df['value']!=df['value_witht_abbr']
cols_l = df.columns.tolist()
df_new = df[cols_l[0:2]+cols_l[-1:]+cols_l[2:-1]]
df = df_new

In [7]:
df['replacement'] = df['replacement'].str.replace(';', '')
df['replacement'] = df['replacement'].str.strip()

In [8]:
df.columns

Index(['Value_ID', 'value', 'IsValueReplaced', 'value_witht_abbr',
       'replacement'],
      dtype='object')

In [9]:
correction_dict = {'Benign BIRADS2 breast nodule': 'Bening breast nodule (BI-RADS)',
 'HT+ DA cavit': 'Hormone therapy',
 'Bilateral hydrosadenitis': 'Hidradenitis suppurativa',
 'SCASEST': 'Acute coronary syndrome',
 'IAM': 'Acute Myocardial Infarction',
 'Alcholic': 'Alcoholic',
 'Hypoacucia': 'Hearing loss',
 'sd IAVD': 'Right ventricle Acute myocardial infarction',
 'Saphenous ischemia': 'Saphenous vein Ischemia'}

In [10]:
df.replace({'replacement':correction_dict}, inplace=True)

In [11]:
# Find Spanish Abbreviations

In [12]:
df_sp_google = pd.read_csv(path+'comorbidities_spanish_english_google.csv', sep=',', error_bad_lines=True, encoding='utf-8')
print (df_sp_google.shape)

df_sp_google_abbr = pd.read_csv(path+'comorbidities_value_abbr_google.csv', sep=',', error_bad_lines=True, encoding='utf-8')
print (df_sp_google.shape)


(5451, 1)
(5451, 1)


In [13]:
df_sp_deep = pd.read_csv(path+'comorbidities_spanish_english_deepl.csv', sep=',', error_bad_lines=True, encoding='utf-8')
print (df_sp_deep.shape)

df_sp_deep_abbr = pd.read_csv(path+'comorbidities_value_abbr_deepl.csv', sep=',', error_bad_lines=True, encoding='utf-8')
print (df_sp_deep.shape)


(5451, 1)
(5451, 1)


In [14]:
df['google'] = df_sp_google['value']
df['deepl'] = df_sp_deep['value']
df['google_abbr'] = df_sp_google_abbr['value_witht_abbr_google']
df['deepl_abbr'] = df_sp_deep_abbr['value_witht_abbr_deepl']

In [15]:
df['google'] = df['google'].str.strip()
df['deepl'] = df['deepl'].str.strip()
df['google_abbr'] = df['google_abbr'].str.strip()
df['deepl_abbr'] = df['deepl_abbr'].str.strip()

In [16]:
def capitalize_(text):
    if text != text:
        text = ''
    if type(text)==str:
        if len(text)>0:
            # return text[0:1].capitalize()+text[1:].lower()
            return ' '.join([i[0:1].upper()+i[1:] if len(i)>1 else i for i in text.split()])
    else:
        return text

df['replacement'] = df['replacement'].apply(capitalize_)
df['google'] = df['google'].apply(capitalize_)
df['deepl'] = df['deepl'].apply(capitalize_)
df['google_abbr'] = df['google_abbr'].apply(capitalize_)
df['deepl_abbr'] = df['deepl_abbr'].apply(capitalize_)

In [17]:
df.head(10)

,Value_ID,value,IsValueReplaced,value_witht_abbr,replacement,google,deepl,google_abbr,deepl_abbr
0,A10000,Gastritis crÛnica,False,Gastritis crÛnica,Chronic Gastritis,Chronic Gastritis,Chronic Gastritis,Chronic Gastritis,Chronic Gastritis
1,A10001,Sarcoma PleomÛrfico Pectoral Mayor,True,Sarcoma PleomÛrfico Pectoral Mayor,Pectoralis Major Pleomorphic Sarcoma,Pectoralis Major Pleomorphic Sarcoma,Pectoralis Major Pectoralis PleomÛrphic Sarcoma,Pectoralis Major Pleomorphic Sarcoma,Pectoralis Major PleomÛrfic Sarcoma
2,A10002,hiperuricemia,False,hiperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia
3,A10003,intervenido pÛlipo colon,False,intervenido pÛlipo colon,Colon Polyp Operated,Colon Plipo Intervened,Intervened PÛlipo Colon,Colon Plipo Intervened,Intervened PÛlipo Colon
4,A10004,intervenido estenosis aorta,False,intervenido estenosis aorta,Aortic Stenosis Operated,Operated Aortic Stenosis,Intervened Aortic Stenosis,Operated Aortic Stenosis,Intervened Aortic Stenosis
5,A10005,Disnea,False,Disnea,Dyspnea,Dyspnoea,Dyspnea,Dyspnoea,Dyspnea
6,A10006,Osteoporosis,False,Osteoporosis,Osteoporosis,Osteoporosis,Osteoporosis,Osteoporosis,Osteoporosis
7,A10007,Histerectomia con doble anexectomia,False,Histerectomia con doble anexectomia,Hysterectomy Double Adnexectomy,Hysterectomy With Double Adnexectomy,Hysterectomy With Double Adnexectomy,Hysterectomy With Double Adnexectomy,Hysterectomy With Double Adnexectomy
8,A10008,hipotiroidismo subclÌnico,False,hipotiroidismo subclÌnico,Subclinical Hypothyroidism,Subclinical Hypothyroidism,Subclinical Hypothyroidism,Subclinical Hypothyroidism,Subclinical Hypothyroidism
9,A10009,carcinoma de prÛstata en 2011,False,carcinoma de prÛstata en 2011,Prostate Carcinoma,Prostate Carcinoma In 2011,PrÛstata Carcinoma In 2011,Prostate Carcinoma In 2011,Prostate Carcinoma In 2011


In [18]:
# Remove stop words and years
def remove_stopwords_numbers(text):
    if text != text:
        text = ''
    return ' '.join([word for word in word_tokenize(str(text)) \
        if word not in all_stopwords and not word.isdigit()])

df['google_withst_num'] = df['google'].apply(remove_stopwords_numbers)
df['deepl_withst_num'] = df['deepl'].apply(remove_stopwords_numbers)
df['google_abbr_withst_num'] = df['google_abbr'].apply(remove_stopwords_numbers)
df['deepl_abbr_withst_num'] = df['deepl_abbr'].apply(remove_stopwords_numbers)

In [19]:
# Calculate Abbreviations
def find_in_english_dict(text):
    for w in word_tokenize(text):
        if Word(w.strip().lower()).spellcheck()[0][1]!=1.0 and len(re.findall(r"[A-Z]{2,}", w))>0 \
            and Word(w.strip().lower()).spellcheck()[0][0]!=w.strip().lower():
            print ("True",w, end='\t')
          # print(w)
            return True
    return False

df['haveAbbr'] = (df['value_witht_abbr'].apply(lambda x : len(re.findall(r"[A-Z]{2,}", x))>0)) & \
    (df['google'].apply(find_in_english_dict))

True SAHS	True BRCA1	True HT	True COPD	True HCV	True HIV	True HIV	True ENOLIC	True SCASEST	True SAOS	True YO	True HIV	True HBV	True BARRET	True ACV	True ACV	True ICTUS	True HLA	True HBV	True HBP	True OSAS	True PYROSIS	True SAHOS	True ENOLISM	True ICTUS	True FXII	True IIA	True TVP	True MII	True ACS	True HH	True ADC	True ADC	True ADC	True MCA	True SAH	True HIATAL	True TEP	True VCH	True HIV	True ALS	True ALLERGY	True TVP	True IAVD	True SCACEST	True ALLERGY	True TEP	True PERONE	True INTOLERANCE	True HIV	True AAS	True ALLERGY	True IBD	True CHIARI	True HIPER	True PTSD	True DVT	True HP	True DM	True CIN	True MITRAL	True COMPULSIVE	True DRIGAS	True TB	True ALLERGY	True ALLERGY	True OSAS	True SAAS	True LSI	True AVB	True AF	True ERC	True CIN	True ILD	True RGE	True FA	True TVP	True FA	True FA	True EOSIN	True CVA	True CR	True POLYPS	True POLYPS	True CVA	True HIV	True HIV	True SAHS	True STENT	True AVC	True TUR	True MCA	True CNS	True STENT	True CRI	True SAHS	True ACV	True IQ	True ACV	True SAHS	True A

In [20]:
# Calculate no of stop words
def stop_words_length(text):
    # print (word_tokenize(str(text)))
    if text != text:
        text = ''
    return len([word for word in word_tokenize(str(text)) if word in all_stopwords])

df['stop_words_google_len'] = df['google'].apply(stop_words_length)
df['stop_words_deepl_len'] = df['deepl'].apply(stop_words_length)
df['stop_words_google_abbr_len'] = df['google_abbr'].apply(stop_words_length)
df['stop_words_deepl_abbr_len'] = df['deepl_abbr'].apply(stop_words_length)

In [21]:
df.to_csv(path_output+'spanish_google_deep_last.csv', index=False, encoding='utf-8')

In [22]:
df.head(5)

,Value_ID,value,IsValueReplaced,value_witht_abbr,replacement,google,deepl,google_abbr,deepl_abbr,google_withst_num,deepl_withst_num,google_abbr_withst_num,deepl_abbr_withst_num,haveAbbr,stop_words_google_len,stop_words_deepl_len,stop_words_google_abbr_len,stop_words_deepl_abbr_len
0,A10000,Gastritis crÛnica,False,Gastritis crÛnica,Chronic Gastritis,Chronic Gastritis,Chronic Gastritis,Chronic Gastritis,Chronic Gastritis,Chronic Gastritis,Chronic Gastritis,Chronic Gastritis,Chronic Gastritis,False,0,0,0,0
1,A10001,Sarcoma PleomÛrfico Pectoral Mayor,True,Sarcoma PleomÛrfico Pectoral Mayor,Pectoralis Major Pleomorphic Sarcoma,Pectoralis Major Pleomorphic Sarcoma,Pectoralis Major Pectoralis PleomÛrphic Sarcoma,Pectoralis Major Pleomorphic Sarcoma,Pectoralis Major PleomÛrfic Sarcoma,Pectoralis Major Pleomorphic Sarcoma,Pectoralis Major Pectoralis PleomÛrphic Sarcoma,Pectoralis Major Pleomorphic Sarcoma,Pectoralis Major PleomÛrfic Sarcoma,False,0,0,0,0
2,A10002,hiperuricemia,False,hiperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia,False,0,0,0,0
3,A10003,intervenido pÛlipo colon,False,intervenido pÛlipo colon,Colon Polyp Operated,Colon Plipo Intervened,Intervened PÛlipo Colon,Colon Plipo Intervened,Intervened PÛlipo Colon,Colon Plipo Intervened,Intervened PÛlipo Colon,Colon Plipo Intervened,Intervened PÛlipo Colon,False,0,0,0,0
4,A10004,intervenido estenosis aorta,False,intervenido estenosis aorta,Aortic Stenosis Operated,Operated Aortic Stenosis,Intervened Aortic Stenosis,Operated Aortic Stenosis,Intervened Aortic Stenosis,Operated Aortic Stenosis,Intervened Aortic Stenosis,Operated Aortic Stenosis,Intervened Aortic Stenosis,False,0,0,0,0


In [23]:
df.tail(5)

,Value_ID,value,IsValueReplaced,value_witht_abbr,replacement,google,deepl,google_abbr,deepl_abbr,google_withst_num,deepl_withst_num,google_abbr_withst_num,deepl_abbr_withst_num,haveAbbr,stop_words_google_len,stop_words_deepl_len,stop_words_google_abbr_len,stop_words_deepl_abbr_len
5446,A15442,AIT en 2017,True,ataque isquémico transitorio en 2017,Ischemic Attack,AIT In 2017,AIT In 2017,Transient Ischemic Attack In 2017,Transient Ischemic Attack In 2017,AIT In,AIT In,Transient Ischemic Attack In,Transient Ischemic Attack In,False,0,0,0,0
5447,A15443,pericarditis en 2013,False,pericarditis en 2013,None,Pericarditis In 2013,Pericarditis In 2013,Pericarditis In 2013,Pericarditis In 2013,Pericarditis In,Pericarditis In,Pericarditis In,Pericarditis In,False,0,0,0,0
5448,A15444,hepatopatia crÛnica con fibrosis e HTP por VHB,True,hepatopatia crÛnica con fibrosis e hipertensión pulmonar por virus hepatitis tipo B,Hepatitis Type B,Chronic Liver Disease With Fibrosis And HBV PHT,Chronic Hepatopathy With Fibrosis And HBV-related PHT,Chronic Hepatopathy With Fibrosis And Pulmonary Hypertension Due To Hepatitis Virus Type B,Chronic Hepatopathy With Fibrosis And Pulmonary Hypertension Due To Hepatitis Virus Type B,Chronic Liver Disease With Fibrosis And HBV PHT,Chronic Hepatopathy With Fibrosis And HBV-related PHT,Chronic Hepatopathy With Fibrosis And Pulmonary Hypertension Due To Hepatitis Virus Type B,Chronic Hepatopathy With Fibrosis And Pulmonary Hypertension Due To Hepatitis Virus Type B,False,0,0,0,0
5449,A15445,enfermedad de crhon,False,enfermedad de crhon,None,Crhon Disease,Crhon's Disease,Crhon's Disease,Crhon's Disease,Crhon Disease,Crhon Disease,Crhon Disease,Crhon Disease,False,0,1,1,1
5450,A15446,intervenido en 3 ocasiones por oclusiÛn intestinal ultima complicada y es portador de fistula entero,False,intervenido en 3 ocasiones por oclusiÛn intestinal ultima complicada y es portador de fistula entero,None,Operated On 3 Occasions For Complicated Last Intestinal Occlusion And Is a Carrier Of An Entire Fistula,Operated 3 Times For Last Complicated Intestinal Occlusion And Is a Carrier Of An Entire Fistula.,Intervened 3 Times For Intestinal Occlusion Last Complicated And Is a Carrier Of a Whole Fistula.,Intervened 3 Times For Intestinal Occlusion Last Complicated And Is a Carrier Of a Whole Fistula.,Operated On Occasions For Complicated Last Intestinal Occlusion And Is Carrier Of An Entire Fistula,Operated Times For Last Complicated Intestinal Occlusion And Is Carrier Of An Entire Fistula .,Intervened Times For Intestinal Occlusion Last Complicated And Is Carrier Of Whole Fistula .,Intervened Times For Intestinal Occlusion Last Complicated And Is Carrier Of Whole Fistula .,False,1,1,2,2


In [24]:
# Train_data
df_MEV = df.dropna(subset=['replacement'])
df_MEV.to_csv(path_output+'train_MEV.csv', index=False, encoding='utf-8')

In [25]:
df_MEV.head(50)

,Value_ID,value,IsValueReplaced,value_witht_abbr,replacement,google,deepl,google_abbr,deepl_abbr,google_withst_num,deepl_withst_num,google_abbr_withst_num,deepl_abbr_withst_num,haveAbbr,stop_words_google_len,stop_words_deepl_len,stop_words_google_abbr_len,stop_words_deepl_abbr_len
0,A10000,Gastritis crÛnica,False,Gastritis crÛnica,Chronic Gastritis,Chronic Gastritis,Chronic Gastritis,Chronic Gastritis,Chronic Gastritis,Chronic Gastritis,Chronic Gastritis,Chronic Gastritis,Chronic Gastritis,False,0,0,0,0
1,A10001,Sarcoma PleomÛrfico Pectoral Mayor,True,Sarcoma PleomÛrfico Pectoral Mayor,Pectoralis Major Pleomorphic Sarcoma,Pectoralis Major Pleomorphic Sarcoma,Pectoralis Major Pectoralis PleomÛrphic Sarcoma,Pectoralis Major Pleomorphic Sarcoma,Pectoralis Major PleomÛrfic Sarcoma,Pectoralis Major Pleomorphic Sarcoma,Pectoralis Major Pectoralis PleomÛrphic Sarcoma,Pectoralis Major Pleomorphic Sarcoma,Pectoralis Major PleomÛrfic Sarcoma,False,0,0,0,0
2,A10002,hiperuricemia,False,hiperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia,False,0,0,0,0
3,A10003,intervenido pÛlipo colon,False,intervenido pÛlipo colon,Colon Polyp Operated,Colon Plipo Intervened,Intervened PÛlipo Colon,Colon Plipo Intervened,Intervened PÛlipo Colon,Colon Plipo Intervened,Intervened PÛlipo Colon,Colon Plipo Intervened,Intervened PÛlipo Colon,False,0,0,0,0
4,A10004,intervenido estenosis aorta,False,intervenido estenosis aorta,Aortic Stenosis Operated,Operated Aortic Stenosis,Intervened Aortic Stenosis,Operated Aortic Stenosis,Intervened Aortic Stenosis,Operated Aortic Stenosis,Intervened Aortic Stenosis,Operated Aortic Stenosis,Intervened Aortic Stenosis,False,0,0,0,0
5,A10005,Disnea,False,Disnea,Dyspnea,Dyspnoea,Dyspnea,Dyspnoea,Dyspnea,Dyspnoea,Dyspnea,Dyspnoea,Dyspnea,False,0,0,0,0
6,A10006,Osteoporosis,False,Osteoporosis,Osteoporosis,Osteoporosis,Osteoporosis,Osteoporosis,Osteoporosis,Osteoporosis,Osteoporosis,Osteoporosis,Osteoporosis,False,0,0,0,0
7,A10007,Histerectomia con doble anexectomia,False,Histerectomia con doble anexectomia,Hysterectomy Double Adnexectomy,Hysterectomy With Double Adnexectomy,Hysterectomy With Double Adnexectomy,Hysterectomy With Double Adnexectomy,Hysterectomy With Double Adnexectomy,Hysterectomy With Double Adnexectomy,Hysterectomy With Double Adnexectomy,Hysterectomy With Double Adnexectomy,Hysterectomy With Double Adnexectomy,False,0,0,0,0
8,A10008,hipotiroidismo subclÌnico,False,hipotiroidismo subclÌnico,Subclinical Hypothyroidism,Subclinical Hypothyroidism,Subclinical Hypothyroidism,Subclinical Hypothyroidism,Subclinical Hypothyroidism,Subclinical Hypothyroidism,Subclinical Hypothyroidism,Subclinical Hypothyroidism,Subclinical Hypothyroidism,False,0,0,0,0
9,A10009,carcinoma de prÛstata en 2011,False,carcinoma de prÛstata en 2011,Prostate Carcinoma,Prostate Carcinoma In 2011,PrÛstata Carcinoma In 2011,Prostate Carcinoma In 2011,Prostate Carcinoma In 2011,Prostate Carcinoma In,PrÛstata Carcinoma In,Prostate Carcinoma In,Prostate Carcinoma In,False,0,0,0,0


In [26]:
print ("Total_Time ({}(in Hrs) : {}(in Mins)".format((time.time()-past_time)/3600.0, (time.time()-past_time)/60.0))

Total_Time (0.45708913213676877(in Hrs) : 27.42534793615341(in Mins)
